In [1]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.4 MB ? eta -:--:--
    --------------------------------------- 0.5/24.4 MB 1.3 MB/s eta 0:00:19
   - -------------------------------------- 0.8/24.4 MB 1.3 MB/s eta 0:00:19
   -- ------------------------------------- 1.3/24.4 MB 1.7 MB/s eta 0:00:14
   -- ------------------------------------- 1.6/24.4 MB 1.6 MB/s eta 0:00:15
   --- ------------------------------------ 1.8/24.4 MB 1.6 MB/s eta 0:00:15
   --- ------------------------------------ 2.4/24.4 MB 1.8 MB/s eta 0:00:13
   ---- ----------------------------------- 2.9/24.4 MB 1.8 MB/s eta 0:00:12
   ------ --------------------------------- 3.7/24.4 MB 2.0 MB/s eta 0:00:11
   ------ --------------------------------- 4.2/24.4 MB 2.0 MB/s eta 0:00:10
   ------- --------


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: C:\Users\humai\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import gensim 
from gensim.models import Word2Vec, keyedvectors

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']

In [ ]:
vec_king.shape

In [3]:
import pandas as pd

df = pd.read_csv('all_kindle_review.csv')

In [4]:
df

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000
...,...,...,...,...,...,...,...,...,...,...,...
11995,11995,2183,B001DUGORO,"[0, 0]",4,Valentine cupid is a vampire- Jena and Ian ano...,"02 28, 2014",A1OKS5Q1HD8WQC,lisa jon jung,jena,1393545600
11996,11996,6272,B002JCSFSQ,"[2, 2]",5,I have read all seven books in this series. Ap...,"05 16, 2011",AQRSPXLNEQAMA,TerryLP,Peacekeepers Series,1305504000
11997,11997,12483,B0035N1V7K,"[0, 1]",3,This book really just wasn't my cuppa. The si...,"07 26, 2013",A2T5QLT5VXOJAK,hwilson,a little creepy,1374796800
11998,11998,3640,B001W1XT40,"[1, 2]",1,"tried to use it to charge my kindle, it didn't...","09 17, 2013",A28MHD2DDY6DXB,"Allison A. Slater ""Gryphon50""",didn't work,1379376000


In [5]:
df = df[['reviewText', 'rating']]

In [6]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [7]:
df.shape

(12000, 2)

In [8]:
## missing values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [9]:
## duplicates
df.duplicated().sum()

np.int64(0)

In [10]:
## check unique values
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [11]:
## check class imbalance
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [12]:
## text preprocessing

In [13]:
df['rating'] = df['rating'].apply(lambda x: 0 if x < 3 else 1)

C:\Users\humai\AppData\Local\Temp\ipykernel_18924\2307663313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['rating'].apply(lambda x: 0 if x < 3 else 1)


In [14]:
df


,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,1
11996,I have read all seven books in this series. Ap...,1
11997,This book really just wasn't my cuppa. The si...,1
11998,"tried to use it to charge my kindle, it didn't...",0


In [15]:
df['rating'].unique()

array([1, 0])

In [16]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [17]:
## lowercase
df['reviewText'] = df['reviewText'].apply(lambda x: x.lower())

C:\Users\humai\AppData\Local\Temp\ipykernel_18924\4230503506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x: x.lower())


In [18]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [19]:
# remove tags, speacial characters
import re
def remove_tags(raw_text):
    return re.sub(re.compile(r'[^a-zA-Z\s]'), '', raw_text)

In [20]:
df['reviewText'] = df['reviewText'].apply(remove_tags)

C:\Users\humai\AppData\Local\Temp\ipykernel_18924\1224342574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(remove_tags)


In [21]:
df.head(8)

,reviewText,rating
0,jace rankin may be short but hes nothing to me...,1
1,great short read i didnt want to put it down ...,1
2,ill start by saying this is the first of four ...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1
5,aislinn is a little girl with big dreams after...,1
6,this has the makings of a good story unfortuna...,0
7,i got this because i like collaborated short s...,1


## Why NOT remove stopwords here?
- Word2Vec learns relationships
- Even stopwords help context learning

In [22]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4') 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\humai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\humai\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [23]:
## Apply Lemmatization
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemma_words(sent):
    return ' '.join([lemmatizer.lemmatize(word) for word in sent.split()])

In [24]:

df['reviewText'] = df['reviewText'].apply(lemma_words)

C:\Users\humai\AppData\Local\Temp\ipykernel_18924\3190671270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lemma_words)


In [25]:
df.head()

,reviewText,rating
0,jace rankin may be short but he nothing to mes...,1
1,great short read i didnt want to put it down s...,1
2,ill start by saying this is the first of four ...,1
3,aggie is angela lansbury who carry pocketbook ...,1
4,i did not expect this type of book to be in li...,1


In [26]:
## Tokenization
## word2vec expects sentences in the list of words not raw sentence
from nltk.tokenize import word_tokenize

print(word_tokenize('my life is beautiful'))

['my', 'life', 'is', 'beautiful']


In [27]:
df['tokens'] = df['reviewText'].apply(word_tokenize)

C:\Users\humai\AppData\Local\Temp\ipykernel_18924\917175080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['reviewText'].apply(word_tokenize)


In [28]:
df.head()

,reviewText,rating,tokens
0,jace rankin may be short but he nothing to mes...,1,"[jace, rankin, may, be, short, but, he, nothin..."
1,great short read i didnt want to put it down s...,1,"[great, short, read, i, didnt, want, to, put, ..."
2,ill start by saying this is the first of four ...,1,"[ill, start, by, saying, this, is, the, first,..."
3,aggie is angela lansbury who carry pocketbook ...,1,"[aggie, is, angela, lansbury, who, carry, pock..."
4,i did not expect this type of book to be in li...,1,"[i, did, not, expect, this, type, of, book, to..."


In [29]:
from gensim.utils import simple_preprocess

print(simple_preprocess('my life is beautiful'))
print(simple_preprocess("i'll start by saying this is the first of four"))

['my', 'life', 'is', 'beautiful']
['ll', 'start', 'by', 'saying', 'this', 'is', 'the', 'first', 'of', 'four']


In [30]:
df[['reviewText', 'tokens']].head()


,reviewText,tokens
0,jace rankin may be short but he nothing to mes...,"[jace, rankin, may, be, short, but, he, nothin..."
1,great short read i didnt want to put it down s...,"[great, short, read, i, didnt, want, to, put, ..."
2,ill start by saying this is the first of four ...,"[ill, start, by, saying, this, is, the, first,..."
3,aggie is angela lansbury who carry pocketbook ...,"[aggie, is, angela, lansbury, who, carry, pock..."
4,i did not expect this type of book to be in li...,"[i, did, not, expect, this, type, of, book, to..."


## Apply Word2Vec

In [31]:
## now for the word2vec convert these tokens into list 
sentences = df['tokens'].tolist()

In [32]:
sentences

[['jace',
  'rankin',
  'may',
  'be',
  'short',
  'but',
  'he',
  'nothing',
  'to',
  'mess',
  'with',
  'a',
  'the',
  'man',
  'who',
  'wa',
  'just',
  'hauled',
  'out',
  'of',
  'the',
  'saloon',
  'by',
  'the',
  'undertaker',
  'know',
  'now',
  'he',
  'a',
  'famous',
  'bounty',
  'hunter',
  'in',
  'oregon',
  'in',
  'the',
  's',
  'who',
  'when',
  'he',
  'shot',
  'the',
  'man',
  'in',
  'the',
  'saloon',
  'just',
  'finished',
  'a',
  'year',
  'long',
  'quest',
  'to',
  'avenge',
  'his',
  'sister',
  'murder',
  'and',
  'is',
  'now',
  'trying',
  'to',
  'figure',
  'out',
  'what',
  'to',
  'do',
  'next',
  'when',
  'the',
  'snottynosed',
  'farm',
  'boy',
  'he',
  'just',
  'rescued',
  'from',
  'a',
  'gang',
  'of',
  'bully',
  'offer',
  'him',
  'money',
  'to',
  'kill',
  'a',
  'man',
  'who',
  'forced',
  'him',
  'off',
  'his',
  'ranch',
  'he',
  'reluctantly',
  'agrees',
  'to',
  'bring',
  'the',
  'man',
  'to',
  '

In [33]:
### vector_size = 100
# - Each word is represented by a 100-dimensional vector

### window = 5
# - How many words left & right Word2Vec looks at

### min_count = 2
# - Ignore words that appear less than 2 times

### epochs = 10
# - Number of times Word2Vec sees the entire dataset

### workers = 4
# - Number of CPU cores used
# - Parallel training

In [34]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    sentences = sentences,
    vector_size=100,
    window = 5,
    min_count=2,
    epochs = 5,
    workers=2
)

In [35]:
# Vocabulary size
len(w2v_model.wv.index_to_key)

# - How many unique words does Word2Vec know?
## 17436


17435

In [36]:
# Vector shape
w2v_model.wv['great'].shape

# - How is ONE word represented numerically?
## 100


(100,)

In [37]:
# Similar words check
w2v_model.wv.similar_by_word('great')

# - Which words are closest in meaning to ‘great’?


[('wonderful', 0.8110778331756592),
 ('nice', 0.8109955191612244),
 ('good', 0.8106967210769653),
 ('fantastic', 0.732461154460907),
 ('fun', 0.6930524706840515),
 ('decent', 0.653606116771698),
 ('cute', 0.6190570592880249),
 ('interesting', 0.6119378209114075),
 ('amazing', 0.602053165435791),
 ('entertaining', 0.5944930911064148)]

In [38]:
import numpy as np

def avg_w2vmodel(tokens, model, vector_size):
    vectors = []

    for word in tokens:
        if word in model.wv:
            vectors.append(model.wv[word])

    if len(vectors) == 0:
        return np.zeros(vector_size)
    
    return np.mean(vectors, axis=0)

In [39]:
x = np.array([
    avg_w2vmodel(tokens, w2v_model, 100)
    for tokens in df['tokens']

])

In [40]:
x.shape


(12000, 100)

In [41]:
# x has reviews y has ratings
# X → features
# y → labels


In [42]:
y = df['rating'].values


In [43]:
x

array([[ 0.26367125,  0.54964244, -0.0419357 , ..., -0.05236194,
         0.16621496, -0.25655136],
       [ 0.3794393 ,  0.56635225,  0.02338326, ...,  0.6201232 ,
         0.6736279 , -0.14247197],
       [ 0.28106448,  0.45441473,  0.02175552, ...,  0.320451  ,
         0.36565134, -0.1436074 ],
       ...,
       [ 0.32734203,  0.5160208 ,  0.08870085, ...,  0.41289595,
         0.34788662, -0.30858856],
       [ 0.19081013,  0.49786153, -0.29067448, ...,  0.26715714,
         0.2874896 , -0.277106  ],
       [ 0.24290444,  0.61472315, -0.00828101, ...,  0.39490098,
         0.3458165 , -0.34528714]], shape=(12000, 100), dtype=float32)

In [44]:
y

array([1, 1, 1, ..., 1, 0, 1], shape=(12000,))

## Apply ML model

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)


In [46]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [48]:
y_pred = clf.predict(X_test)

In [49]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))


0.8033333333333333
